# Robotic Arm



A [Robotic arm](https://en.wikipedia.org/wiki/Robotic_arm) is a type of mechanical arm, usually programmable, with similar characteristics of a human arm. The purpose of the robot is to replace the human in difficult and harmful tasks e.g., Industrial production, processing and manufacturing roles. Any task in which extremely precise, fast and repeatable movements are required as well as to automate the process of doing the daily task that humans typically do.

The [arm](https://www.universal-robots.com/in/blog/working-of-robotic-arm/#:~:text=Robotic%20arms%20refer%20to%20a,degrees%20of%20freedom%20of%20movement) 
contains multiple joints that act as axes that enable a degree of movement. Six-axis robots are the standard and have a full range of motion. It's structured with a shoulder, an elbow and a wrist and even finger tips. There are more robots with more than 6 joints. The different joints makes it possible for the robot to move freely in certain required degrees. Likewise, the larger quantity of rotary joints a robotic arm features, the more freedom of movement it has. To manipulate, we use the links that are connected by joints allowing rotational motion or translational displacement.

![Model](https://www.mathworks.com/help/examples/fuzzy/win64/invkine_codepad_01.png)

In additional to [rotary joints](https://www.intel.in/content/www/in/en/robotics/robotic-arm.html#:~:text=In%20addition%20to%20rotary%20joints,power%20systems%2C%20and%20software%20components.), robotic arm components include the robot controller, an end-of-arm tool, actuators, sensors, vision systems, power systems, and software components.

# Rotation with the help of Rotation Matrices

A Robot Arm has position and orientation to be able to adapt to movements and perform its job effectively. It is the position of something in relation to its surroundings with different diversity. 

A rotation is represented in a matrix. In Cartesian space ![Cartesian space](https://www.tthk.ee/inlearcs/wp-content/uploads/2020/09/7.gif)

a robot orientation is decided by a combinations of rotations in X, Y and Z direction, and we can have a 3-by-3 rotation matrix for each orientation.

$$ R = \begin{bmatrix}
    \ r11 & \ r12 & \ r13 \\
    \ r21 & \ r22 & \ r23 \\
    \ r31 & \ r32 & \ r33
    \end{bmatrix}
$$

The __axis-angle__ representation is used for robot __orientation__. _Let's say_, we want the angle to point to a _certain_ __direction__, we use a so called __direction vector__. Around the vector, an orientation can be rotated by a certain angle, _theta_. As a result of rotation, we can have a different orientation. The axis-angle representation is useful for robotics calculation such as kinematics and dynamics.

$$ \dotsm U_Vector = \begin{bmatrix}
            \ ux \\
            \ uy \\
            \ u\approx \\
            \end{bmatrix}